In [1]:
from gurobi import *
import pandas as pd
import csv

In [2]:
Data_all = pd.read_excel("Data.xlsx",sheet_name="Table")
Time_all = pd.read_excel("Data.xlsx", sheet_name = "Time")

Data_all.set_index(['Carrier','Arrival'], inplace=True)
Time_all.set_index(['Index','Airport'], inplace=True)

Data = Data_all 
Time = Time_all

m = Model()
removed = ['Frontier Airlines', 'Hawaiian Airlines Network', 'Allegiant Air','Alaska Airlines Network']
# 4 airlines were removed due to them not having data for all the points (ie they don't fly from all the locations or to SFO)

Using license file /Users/elizabethhocking/gurobi.lic
Academic license - for non-commercial use only


In [3]:
## user input validation function
def get_valid_input(inp_str, valid_inp):
    valid_inp = list(map(lambda x: str(x), valid_inp))
    inp = input(inp_str)
    while inp not in valid_inp:
        inp = input(inp_str)
    return inp

# Stressor weights input
do_survey = get_valid_input(
        'Would you like to input your travel stressors? (Y/N)\nif you choose not to, default values will be used.',
        ['Y', 'N'])

if do_survey == 'Y':
    print('On a scale of 1-10, how stressed would you be if the following occured:')
    stress_cancel = get_valid_input('your flight is cancelled', list(range(1, 11)))
    stress_divert = get_valid_input('your flight is diverted from your destination', list(range(1, 11)))
    stress_late = get_valid_input('your flight is late', list(range(1,11)))
else:
    stress_cancel = 10
    stress_divert = 5
    stress_late = 2


Would you like to input your travel stressors? (Y/N)
if you choose not to, default values will be used.Y
On a scale of 1-10, how stressed would you be if the following occured:
your flight is cancelled10
your flight is diverted from your destination7
your flight is late4


In [4]:
# add a constraint deciding which airport
Data.index
Airport = ['EWR', 'JFK', 'LGA']

Airline = []
for i in range(0,11):
    if Data.index[i][0] not in removed:
        Airline.append(Data.index[i][0])

## ---- Variables ----

# variable = 1 if we fly from that airport, 0 if we don't 
airport = {}
for a in Airport:
    airport[a] = m.addVar(vtype=GRB.BINARY)
    
# variable = 1 if we take that carrier, 0 if we don't
airline = {} #airline
for a_line in Airline:
    airline[a_line] = m.addVar(vtype=GRB.BINARY)

# flight timing 
time = {}
for i in range(1, 19):
    time[i] = m.addVar(vtype=GRB.BINARY)

e = {}
for a in Airport:
    e[a] = {}
    for a_line in Airline:
        e[a][a_line] = m.addVar(vtype=GRB.BINARY)
    
schedule = {}
for a in Airport:
    schedule[a] = {}
    for r in range(1, 19):
        schedule[a][r] = m.addVar(vtype=GRB.BINARY)

# ---- Constraints ----

# we should only fly from one airport
m.addConstr(quicksum(airport[a] for a in Airport) == 1) #constraint 8
m.addConstr(quicksum(airline[a_line] for a_line in Airline) == 1) #constraint 10 fly with one airline
m.addConstr(quicksum(time[i] for i in range(1,19)) == 1) #constraint 9 fly only ine time

#constraints 6
for a in Airport:
    for a_line in Airline:
        m.addConstr(e[a][a_line] <= airport[a])
        m.addConstr(e[a][a_line] <= airline[a_line])
        m.addConstr(e[a][a_line] >= airport[a] + airline[a_line] - 1)
        m.addConstr(e[a][a_line] >= 0)
        m.addConstr(e[a][a_line] <= 1)

#constraints 7
for a in Airport:
    for r in range(1, 19):
        m.addConstr(schedule[a][r] <= airport[a])
        m.addConstr(schedule[a][r] <= time[r])
        m.addConstr(schedule[a][r] >= airport[a] + time[r] - 1)
        m.addConstr(schedule[a][r] <= 1)
        m.addConstr(schedule[a][r] >= 0)

In [5]:
# objective function 

#obj cancelled 3
obj_cancel = stress_cancel * quicksum(e[a][a_line] * Data.loc[a_line, a]["% Cancelled"]
                                      for a in Airport for a_line in Airline)

#4
obj_divert = stress_divert * quicksum(e[a][a_line] * Data.loc[a_line, a]["% Diverted"]
                                       for a in Airport for a_line in Airline)
# 1
obj_late = stress_late * quicksum(e[a][a_line] * (1 - Data.loc[a_line, a]["% On Time"]) 
                                   for a in Airport for a_line in Airline)
# 2
obj_sched = stress_late * quicksum(schedule[a][r] * (1 - Time.loc[r, a]["% On time"]) 
                                   for a in Airport for r in range(1,18))

m.setObjective(obj_cancel + obj_divert + obj_late, GRB.MINIMIZE)

In [6]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 378 rows, 103 columns and 703 nonzeros
Model fingerprint: 0xcfa7da84
Variable types: 0 continuous, 103 integer (103 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.7352000
Presolve removed 150 rows and 0 columns
Presolve time: 0.00s
Presolved: 228 rows, 103 columns, 553 nonzeros
Variable types: 0 continuous, 103 integer (103 binary)

Root relaxation: objective 0.000000e+00, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   11    1.73520    0.00000   100%     -    0s
H    0     0                       1.5581000    0.00000   100%     -    0s
H    0     0                       1.2991000    0.00000   

In [7]:
airport_sol = []
airline_sol = []
time_sol = []

sol = []
for a in Airport:
    val = airport[a].x
    if(val == 1):
        sol.append(('airport', a))
    airport_sol.append((a, val))

for a_line in Airline:
    val = airline[a_line].x
    if(val == 1):
        sol.append(('airline', a_line))
    airline_sol.append((a_line, val))

def timeformat(i):
    if i == 18:
        return "23:00-6:00"
    else:
        return str(i + 5) + ":00-" + str(i + 6) + ":00"

for r in range(1,19):
    val = time[r].x
    if(val == 1):
        timeslot = timeformat(r)
        sol.append(('time', timeslot))
    time_sol.append((r, val))    

print(airport_sol)
print(airline_sol)
print(time_sol)

obj = m.getObjective()
obj_val = obj.getValue()
print(obj_val)

sol.append(('objective value', obj_val))

##output
with open('sol.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for s in sol:
        wr.writerow([s[0], s[1]])

[('EWR', -0.0), ('JFK', 1.0), ('LGA', -0.0)]
[('American Airlines Network', 0.0), ('Delta Air Line Network', 1.0), ('Jetblue Airways', 0.0), ('Southwest Airlines', 0.0), ('Spirit Airlines', 0.0), ('United Airlines Network', 0.0), ('Total', 0.0)]
[(1, 1.0), (2, -0.0), (3, -0.0), (4, -0.0), (5, -0.0), (6, -0.0), (7, -0.0), (8, -0.0), (9, -0.0), (10, -0.0), (11, -0.0), (12, -0.0), (13, -0.0), (14, -0.0), (15, -0.0), (16, -0.0), (17, -0.0), (18, -0.0)]
0.5172
